In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import mysql.connector
from datetime import datetime, timedelta

In [2]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'abcd1234',
    'database': 'iitr',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [3]:
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()


In [4]:


query = "SELECT * FROM stockdata2024"

# Execute the query and load data into a pandas DataFrame
dataframe = pd.read_sql(query, connection)

# Close the database connection
connection.close()




/var/folders/vj/lr0smv8s7cjfn1_wjtdkj1c80000gn/T/ipykernel_27707/2436113397.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, connection)


In [5]:
import numpy as np
dataframe = dataframe.replace('-', np.nan)

In [7]:
dataframe['DATE1'] = pd.to_datetime(dataframe['DATE1'])

# Extract distinct dates from the 'date' column
distinct_dates = dataframe['DATE1'].drop_duplicates()

# Format the dates as "DDMMYYYY"
dates = distinct_dates.dt.strftime('%d%m%Y').tolist()

# Display the list of formatted dates
print(dates)
print(len(dates))

['01012024', '02012024', '03012024', '04012024', '05012024', '08012024', '09012024', '10012024', '11012024', '12012024', '15012024', '16012024', '17012024', '18012024', '19012024', '20012024', '23012024', '24012024', '25012024', '29012024', '30012024', '31012024', '01022024', '02022024', '05022024', '06022024', '07022024', '08022024', '09022024', '12022024', '13022024', '14022024', '15022024', '16022024', '19022024', '20022024', '21022024', '22022024', '23022024', '26022024', '27022024', '28022024', '29022024', '01032024', '04032024', '05032024', '06032024', '07032024', '11032024', '12032024', '13032024', '14032024', '15032024', '18032024', '19032024', '20032024', '21032024', '22032024', '26032024', '27032024', '28032024', '01042024', '02042024', '03042024', '04042024', '05042024', '08042024', '09042024', '10042024', '12042024', '15042024', '16042024', '18042024', '19042024', '22042024', '23042024', '24042024', '25042024', '26042024', '29042024', '30042024', '02052024', '03052024', '06

In [37]:
print(dates.index('01072024'))

122


In [36]:
# final_ans6 = pd.DataFrame()
# final_ans7 = pd.DataFrame()

# for i in range(20,21):
#     # Select 4 consecutive dates
#     selected_dates = dates[i:i + 4]
#     # Filter the DataFrame to include only the selected dates
#     selected_dates_datetime = [pd.to_datetime(date, format='%d%m%Y') for date in selected_dates]
    
#     # Filter the DataFrame
#     df= dataframe[dataframe['DATE1'].isin(selected_dates_datetime)]
#     ans6,ans7= find_pattern(df)
    
#     final_ans6 = pd.concat([final_ans6, ans6], ignore_index=True).drop_duplicates()
#     final_ans7 = pd.concat([final_ans7, ans7], ignore_index=True).drop_duplicates()

# Write to CSV
#final_ans6.to_csv('filtered_stocks_6cond.csv', index=False)
#final_ans7.to_csv('filtered_stocks_7cond.csv', index=False)


In [38]:

    # Select 4 consecutive dates
selected_dates = dates[122:122 + 4]
    # Filter the DataFrame to include only the selected dates
selected_dates_datetime = [pd.to_datetime(date, format='%d%m%Y') for date in selected_dates]
    
    # Filter the DataFrame
df= dataframe[dataframe['DATE1'].isin(selected_dates_datetime)]
#    ans6,ans7= find_pattern(df)

In [39]:
filtered_df = df[df['SYMBOL'] == 'NBCC']

In [40]:
df = filtered_df.reset_index(drop=True).copy()
required_columns = ['SYMBOL', 'CLOSE_PRICE', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'TTL_TRD_QNTY', 'DELIV_PER']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise KeyError(f"Missing columns: {', '.join(missing_columns)}")
df['same script'] = 0
for i in range(2, len(df)):
    if df['SYMBOL'][i] == df['SYMBOL'][i-1] and df['SYMBOL'][i]== df['SYMBOL'][i-2]:
         df.at[i, 'same script'] = 1

In [43]:
df['%price change'] = None
for i in range(0, len(df)):
    df.at[i, '%price change'] = 100*(((df['CLOSE_PRICE'][i])-(df['PREV_CLOSE'][i]))/df['PREV_CLOSE'][i])


In [44]:
df['%3 consecutive chng >2.5'] = 0
for i in range(2, len(df)):
    if df['%price change'][i]>=2.5 and df['%price change'][i-1]>=2.5 and df['%price change'][i-2]>2.5:
        df.at[i, '%3 consecutive chng >2.5'] = 1
 
df['body'] = 0
for i in range(0, len(df)):
    df.at[i,'body'] = (100 * np.abs(df['CLOSE_PRICE'][i] - df['OPEN_PRICE'][i]) / (df['HIGH_PRICE'][i] - df['LOW_PRICE'][i]))
    
df['body>65%'] = 0
for i in range(2, len(df)):
    if df['body'][i]>=65 and df['body'][i-1]>=65 and df['body'][i-2]>65:
        df.at[i, 'body>65%'] = 1
print(df)

  SYMBOL SERIES      DATE1  PREV_CLOSE  OPEN_PRICE  HIGH_PRICE  LOW_PRICE  \
0   NBCC     EQ 2024-07-01      156.96      156.98      160.48     155.80   
1   NBCC     EQ 2024-07-02      156.60      157.30      163.20     156.32   
2   NBCC     EQ 2024-07-03      162.28      163.50      170.41     161.30   
3   NBCC     EQ 2024-07-04      169.03      171.52      189.90     170.72   

   LAST_PRICE  CLOSE_PRICE  AVG_PRICE  ...  DELIV_PER  same script  \
0      156.63       156.60     157.79  ...      38.40            0   
1      162.80       162.28     160.08  ...      36.60            0   
2      168.72       169.03     166.91  ...      34.47            1   
3      186.50       186.68     180.09  ...      24.69            1   

   prev close<today close prev open<today open  Prev Day Vol< Today Vol  \
0                       0                    0                        0   
1                       1                    1                        1   
2                       1             

/var/folders/vj/lr0smv8s7cjfn1_wjtdkj1c80000gn/T/ipykernel_27707/2636461724.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.11965811965806' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i,'body'] = (100 * np.abs(df['CLOSE_PRICE'][i] - df['OPEN_PRICE'][i]) / (df['HIGH_PRICE'][i] - df['LOW_PRICE'][i]))


In [45]:
df['prev close<today close'] = 0
for i in range(0, len(df)):
    if df['PREV_CLOSE'][i]<df['CLOSE_PRICE'][i]:
        df.at[i, 'prev close<today close'] = 1
    
df['prev open<today open'] = 0
for i in range(1, len(df)):
    if df['OPEN_PRICE'][i-1]<df['OPEN_PRICE'][i]:
        df.at[i, 'prev open<today open'] = 1

df['Prev Day Vol< Today Vol']=0
for i in range(1, len(df)):
    if df['TTL_TRD_QNTY'][i-1]<df['TTL_TRD_QNTY'][i]:
        df.at[i, 'Prev Day Vol< Today Vol'] = 1
df['DELIV_PER'] = pd.to_numeric(df['DELIV_PER'], errors='coerce')

df['Prev day del<Today Del'] = 0
for i in range(1, len(df)):
    # Check if both previous and current values are not NaN
    if not pd.isna(df.at[i-1, 'DELIV_PER']) and not pd.isna(df.at[i, 'DELIV_PER']):
        # Compare previous day's DELIV_PER with today's DELIV_PER
        if df.at[i-1, 'DELIV_PER'] < df.at[i, 'DELIV_PER']:
            df.at[i, 'Prev day del<Today Del'] = 1

print(df.iloc[:, -10:])

   DELIV_PER  same script  prev close<today close  prev open<today open  \
0      38.40            0                       0                     0   
1      36.60            0                       1                     1   
2      34.47            1                       1                     1   
3      24.69            1                       1                     1   

   Prev Day Vol< Today Vol  Prev day del<Today Del %price change  \
0                        0                       0     -0.229358   
1                        1                       0      3.627075   
2                        1                       0      4.159477   
3                        1                       0     10.441933   

   %3 consecutive chng >2.5       body  body>65%  
0                         0   8.119658         0  
1                         0  72.383721         0  
2                         0  60.702525         0  
3                         1  79.040667         0  


In [46]:
df['sum_without_del']=0
for i in range(2, len(df)):
    df.at[i, 'sum_without_del'] = df['same script'][i]+df['%3 consecutive chng >2.5'][i]+df['prev close<today close'][i]+df['prev open<today open'][i]+ df['body>65%'][i]+df['Prev Day Vol< Today Vol'][i]
res6=(df[df['sum_without_del']>=6])

print(df.iloc[:, -10:])

   same script  prev close<today close  prev open<today open  \
0            0                       0                     0   
1            0                       1                     1   
2            1                       1                     1   
3            1                       1                     1   

   Prev Day Vol< Today Vol  Prev day del<Today Del %price change  \
0                        0                       0     -0.229358   
1                        1                       0      3.627075   
2                        1                       0      4.159477   
3                        1                       0     10.441933   

   %3 consecutive chng >2.5       body  body>65%  sum_without_del  
0                         0   8.119658         0                0  
1                         0  72.383721         0                0  
2                         0  60.702525         0                4  
3                         1  79.040667         0                5 

In [26]:
print(res6.iloc[:,-8:])

   %3 consecutive chng >2.5       body  body>65%  prev close<today close  \
3                         1  77.042802         1                       1   

   prev open<today open  Prev Day Vol< Today Vol  Prev day del<Today Del  \
3                     1                        1                       0   

   sum_without_del  
3                6  


In [72]:
def find_pattern(daf):
    df = daf.reset_index(drop=True).copy()
    required_columns = ['SYMBOL', 'CLOSE_PRICE', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'TTL_TRD_QNTY', 'DELIV_PER']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"Missing columns: {', '.join(missing_columns)}")
    df['same script'] = 0
    for i in range(2, len(df)):
        if df['SYMBOL'][i] == df['SYMBOL'][i-1] == df['SYMBOL'][i-2]:
             df.at[i, 'same script'] = 1
             
    df['%price change'] = None
    for i in range(0, len(df)):
        df.at[i, '%price change'] = 100*(((df['CLOSE_PRICE'][i])-(df['PREV_CLOSE'][i]))/df['PREV_CLOSE'][i])

    df['%3 consecutive chng >2.5'] = 0
    for i in range(2, len(df)):
        if df['%price change'][i]>=2.5 and df['%price change'][i-1]>=2.5 and df['%price change'][i-2]>2.5:
            df.at[i, '%3 consecutive chng >2.5'] = 1
 
    df['body'] = 0
    for i in range(0, len(df)):
        df.at[i,'body'] = (100 * np.abs(df['CLOSE_PRICE'][i] - df['OPEN_PRICE'][i]) / (df['HIGH_PRICE'][i] - df['LOW_PRICE'][i]))
    
    df['body>65%'] = 0
    for i in range(2, len(df)):
        if df['body'][i]>=65 and df['body'][i-1]>=65 and df['body'][i-2]>65:
            df.at[i, 'body>65%'] = 1

    df['prev close<today close'] = 0
    for i in range(0, len(df)):
        if df['PREV_CLOSE'][i]<df['CLOSE_PRICE'][i]:
            df.at[i, 'prev close<today close'] = 1
    
    df['prev open<today open'] = 0
    for i in range(1, len(df)):
        if df['OPEN_PRICE'][i-1]<df['OPEN_PRICE'][i]:
            df.at[i, 'prev open<today open'] = 1

    df['Prev Day Vol< Today Vol']=0
    for i in range(1, len(df)):
        if df['TTL_TRD_QNTY'][i-1]<df['TTL_TRD_QNTY'][i]:
            df.at[i, 'Prev Day Vol< Today Vol'] = 1
    df['DELIV_PER'] = pd.to_numeric(df['DELIV_PER'], errors='coerce')

    df['Prev day del<Today Del'] = 0
    for i in range(1, len(df)):
    # Check if both previous and current values are not NaN
        if not pd.isna(df.at[i-1, 'DELIV_PER']) and not pd.isna(df.at[i, 'DELIV_PER']):
        # Compare previous day's DELIV_PER with today's DELIV_PER
            if df.at[i-1, 'DELIV_PER'] < df.at[i, 'DELIV_PER']:
                df.at[i, 'Prev day del<Today Del'] = 1
    
    df['sum']=0
    for i in range(2, len(df)):
        df.at[i, 'sum'] = df['same script'][i]+df['%3 consecutive chng >2.5'][i]+df['prev close<today close'][i]+df['prev open<today open'][i]+ df['body>65%'][i]+df['Prev Day Vol< Today Vol'][i]+df['Prev day del<Today Del'][i]
    res7=df[df['sum'] >= 7]

    df['sum_without_del']=0
    for i in range(2, len(df)):
        df.at[i, 'sum_without_del'] = df['same script'][i]+df['%3 consecutive chng >2.5'][i]+df['prev close<today close'][i]+df['prev open<today open'][i]+ df['body>65%'][i]+df['Prev Day Vol< Today Vol'][i]
    res6=(df[df['sum_without_del']>=6])
    return res6,res7

